In [2]:
def generate_equations(shuffle=True, max_count=None):
    """
    Generates all possible math equations given the global configuration.
    If max_count is given, returns that many at most. If shuffle is True,
    the equation will be generated in random order.
    """
    # Generate all possible unique sets of numbers
    number_permutations = itertools.permutations(
        range(MIN_NUMBER, MAX_NUMBER + 1), 2
    )
    operators = ['+','-','/','*']
    length = [3,5]


    # Shuffle if required. The downside is we need to convert to list first
    if shuffle:
        number_permutations = list(number_permutations)
        random.shuffle(number_permutations)

    # If a max_count is given, use itertools to only look at that many items
    if max_count is not None:
        number_permutations = itertools.islice(number_permutations, max_count)

    # Build an equation string for each and yield to caller
    for x, y in number_permutations:
      l = random.randint(0,1)
      if length[l] == 3:
        index = random.randint(0,3)
        yield '{} {} {}'.format(x,operators[index] ,y)
      elif length[l] == 5:
        index1 = random.randint(0,3)
        index2 = random.randint(0,3)
        yield '{} {} {} {} {}'.format(x,operators[index1] ,y, operators[index2],y/2,x/4)

In [3]:
def equations_to_x_y(equations, n):
    """
    Given a list of equations, converts them to one-hot vectors to build
    two data matrixes x and y.
    """
    x = np.zeros(
        (n, MAX_EQUATION_LENGTH, N_FEATURES), dtype=np.bool
    )
    y = np.zeros(
        (n, MAX_RESULT_LENGTH, N_FEATURES), dtype=np.bool
    )

    # Get the first n_test equations and convert to test vectors
    for i, equation in enumerate(itertools.islice(equations, n)):
        print("equation is",equation,end=" ")
        try:
            result = eval(equation)
            result = int(result)
            result = str(result)
            print("result is", result)
        except:
            continue

        # Pad the result with spaces
        result = ' ' * (MAX_RESULT_LENGTH - 1 - len(result)) + result

        # We end each sequence in a sequence-end-character:
        equation += '\0'
        result += '\0'

        for t, char in enumerate(equation):
            x[i, t, CHAR_TO_INDEX[char]] = 1

        for t, char in enumerate(result):
            y[i, t, CHAR_TO_INDEX[char]] = 1

    return x, y


def build_dataset():
    """
    Generates equations based on global config, splits them into train and test
    sets, and returns (x_test, y_test, x_train, y_train).
    """
    generator = generate_equations(max_count=N_EXAMPLES)

    # Split into training and test set based on SPLIT:
    n_test = round(SPLIT * N_EXAMPLES)
    n_train = N_EXAMPLES - n_test

    x_test, y_test = equations_to_x_y(generator, n_test)
    x_train, y_train = equations_to_x_y(generator, n_train)

    return x_test, y_test, x_train, y_train

In [4]:
def print_example_predictions(count, model, x_test, y_test):
    """
    Print some example predictions along with their target from the test set.
    """
    print('Examples:')

    # Pick some random indices from the test set
    prediction_indices = np.random.choice(
        x_test.shape[0], size=count, replace=False
    )
    # Get a prediction of each
    predictions = model.predict(x_test[prediction_indices, :])

    for i in range(count):
        print('{} = {}   (expected: {})'.format(
            one_hot_to_string(x_test[prediction_indices[i]]),
            one_hot_to_string(predictions[i]),
            one_hot_to_string(y_test[prediction_indices[i]]),
        ))

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, Dense, Activation
from tensorflow.keras.layers import TimeDistributed, Bidirectional
from tensorflow.keras.optimizers import Adam

def build_model():
    """
    Builds and returns the model based on the global config.
    """
    input_shape = (MAX_EQUATION_LENGTH, N_FEATURES)

    model = Sequential()

    # Encoder:
    model.add(Bidirectional(LSTM(20), input_shape=input_shape))
    # The RepeatVector-layer repeats the input n times
    model.add(RepeatVector(MAX_RESULT_LENGTH))

    # Decoder:
    model.add(Bidirectional(LSTM(20, return_sequences=True)))

    model.add(TimeDistributed(Dense(N_FEATURES)))
    model.add(Activation('softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.01),
        metrics=['accuracy'],
    )

    return model

In [6]:
import numpy as np

CHARS = [str(n) for n in range(10)] + ['+', '-', '*','/','.',' ', '\0']
CHAR_TO_INDEX = {i: c for c, i in enumerate(CHARS)}
INDEX_TO_CHAR = {c: i for c, i in enumerate(CHARS)}

def one_hot_to_index(vector):
    if not np.any(vector):
        return -1

    return np.argmax(vector)

def one_hot_to_char(vector):
    index = one_hot_to_index(vector)
    if index == -1:
        return ''

    return INDEX_TO_CHAR[index]

def one_hot_to_string(matrix):
    return ''.join(one_hot_to_char(vector) for vector in matrix)

In [7]:
MIN_NUMBER = 0
MAX_NUMBER = 999

MAX_N_EXAMPLES = (MAX_NUMBER - MIN_NUMBER) ** 2
N_EXAMPLES = 30000
N_FEATURES = len(CHARS)
MAX_NUMBER_LENGTH_LEFT_SIDE = len(str(MAX_NUMBER))
MAX_NUMBER_LENGTH_RIGHT_SIDE = MAX_NUMBER_LENGTH_LEFT_SIDE + 1
MAX_EQUATION_LENGTH = 21
MAX_RESULT_LENGTH = 12

SPLIT = .1
EPOCHS = 300
BATCH_SIZE = 256

RANDOM_SEED = 1

In [8]:
import random
import itertools
random.seed(RANDOM_SEED)

x_test, y_test, x_train, y_train = build_dataset()

/var/folders/k7/dvm_qjfs0wd10b19j7jnj8rw0000gn/T/ipykernel_19953/107997849.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (n, MAX_EQUATION_LENGTH, N_FEATURES), dtype=np.bool
/var/folders/k7/dvm_qjfs0wd10b19j7jnj8rw0000gn/T/ipykernel_19953/107997849.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (n, MAX_RESULT_LENGTH, N_FEATURES), dtype=n

equation is 502 - 42 - 21.0 result is 439
equation is 983 / 961 result is 1
equation is 856 * 328 * 164.0 result is 46045952
equation is 978 + 729 * 364.5 result is 266698
equation is 829 * 433 result is 358957
equation is 253 / 98 result is 2
equation is 230 + 105 / 52.5 result is 232
equation is 139 * 607 / 303.5 result is 278
equation is 590 / 949 + 474.5 result is 475
equation is 377 * 918 result is 346086
equation is 700 / 52 result is 13
equation is 482 / 298 result is 1
equation is 397 / 624 result is 0
equation is 320 * 307 - 153.5 result is 98086
equation is 305 * 483 result is 147315
equation is 782 * 881 * 440.5 result is 303478951
equation is 681 * 278 * 139.0 result is 26315202
equation is 112 / 708 result is 0
equation is 182 + 614 result is 796
equation is 187 / 21 * 10.5 result is 93
equation is 949 - 655 * 327.5 result is -213563
equation is 925 + 687 result is 1612
equation is 232 - 246 / 123.0 result is 230
equation is 907 + 366 - 183.0 result is 1090
equation is 746

equation is 203 / 838 + 419.0 result is 419
equation is 949 / 874 result is 1
equation is 609 + 120 result is 729
equation is 559 * 342 result is 191178
equation is 877 / 239 result is 3
equation is 326 - 293 result is 33
equation is 783 / 674 result is 1
equation is 926 / 558 result is 1
equation is 400 * 918 result is 367200
equation is 899 - 987 - 493.5 result is -581
equation is 598 - 631 result is -33
equation is 1 - 775 * 387.5 result is -300311
equation is 9 / 891 result is 0
equation is 338 * 920 - 460.0 result is 310500
equation is 793 - 460 result is 333
equation is 710 - 373 * 186.5 result is -68854
equation is 253 / 836 + 418.0 result is 418
equation is 81 - 648 * 324.0 result is -209871
equation is 709 / 789 result is 0
equation is 626 / 265 * 132.5 result is 313
equation is 657 + 823 result is 1480
equation is 525 + 497 result is 1022
equation is 242 / 859 result is 0
equation is 779 / 488 result is 1
equation is 950 - 877 result is 73
equation is 574 / 427 - 213.5 result

equation is 12 - 304 result is -292
equation is 660 + 281 / 140.5 result is 662
equation is 98 - 728 result is -630
equation is 214 / 270 result is 0
equation is 549 - 931 result is -382
equation is 592 / 486 result is 1
equation is 820 / 623 / 311.5 result is 0
equation is 745 / 388 / 194.0 result is 0
equation is 341 + 220 / 110.0 result is 343
equation is 895 * 717 result is 641715
equation is 322 / 141 - 70.5 result is -68
equation is 394 * 931 result is 366814
equation is 830 + 735 - 367.5 result is 1197
equation is 239 + 405 + 202.5 result is 846
equation is 49 * 251 * 125.5 result is 1543524
equation is 845 / 165 result is 5
equation is 881 / 165 result is 5
equation is 675 - 727 * 363.5 result is -263589
equation is 663 * 855 result is 566865
equation is 996 * 639 result is 636444
equation is 505 + 255 result is 760
equation is 480 / 482 result is 0
equation is 457 - 235 result is 222
equation is 742 * 764 * 382.0 result is 216551216
equation is 173 + 826 result is 999
equation

equation is 330 + 28 * 14.0 result is 722
equation is 410 + 628 result is 1038
equation is 821 + 684 * 342.0 result is 234749
equation is 291 / 895 result is 0
equation is 243 - 232 result is 11
equation is 685 - 763 / 381.5 result is 683
equation is 532 / 379 * 189.5 result is 266
equation is 804 / 716 - 358.0 result is -356
equation is 281 * 100 * 50.0 result is 1405000
equation is 516 - 943 - 471.5 result is -898
equation is 712 * 946 + 473.0 result is 674025
equation is 537 - 797 result is -260
equation is 579 / 465 result is 1
equation is 85 + 561 + 280.5 result is 926
equation is 996 - 650 * 325.0 result is -210254
equation is 682 * 544 * 272.0 result is 100914176
equation is 713 + 248 - 124.0 result is 837
equation is 15 * 73 result is 1095
equation is 555 / 463 result is 1
equation is 504 + 979 result is 1483
equation is 926 / 939 result is 0
equation is 387 - 419 - 209.5 result is -241
equation is 313 * 749 result is 234437
equation is 397 + 673 result is 1070
equation is 439 

...

equation is 108 * 545 result is 58860
equation is 886 + 66 / 33.0 result is 888
equation is 384 + 109 result is 493
equation is 937 + 916 - 458.0 result is 1395
equation is 145 / 520 result is 0
equation is 981 * 907 + 453.5 result is 890220
equation is 399 - 535 / 267.5 result is 397
equation is 245 * 4 - 2.0 result is 978
equation is 139 / 665 result is 0
equation is 961 / 242 result is 3
equation is 554 + 161 result is 715
equation is 765 + 534 result is 1299
equation is 58 + 964 result is 1022
equation is 826 * 353 + 176.5 result is 291754
equation is 337 - 2 / 1.0 result is 335
equation is 611 + 835 result is 1446
equation is 935 / 689 - 344.5 result is -343
equation is 177 / 340 * 170.0 result is 88
equation is 906 - 562 result is 344
equation is 353 / 258 - 129.0 result is -127
equation is 815 + 535 / 267.5 result is 817
equation is 617 - 612 result is 5
equation is 629 - 945 * 472.5 result is -445883
equation is 811 * 491 * 245.5 result is 97758345
equation is 265 - 544 result 

equation is 564 / 245 - 122.5 result is -120
equation is 131 + 668 + 334.0 result is 1133
equation is 37 * 234 / 117.0 result is 74
equation is 930 * 789 - 394.5 result is 733375
equation is 172 * 924 result is 158928
equation is 349 / 709 result is 0
equation is 744 - 77 + 38.5 result is 705
equation is 638 / 630 result is 1
equation is 521 / 82 result is 6
equation is 378 / 192 + 96.0 result is 97
equation is 811 + 687 / 343.5 result is 813
equation is 455 * 792 result is 360360
equation is 79 * 271 / 135.5 result is 158
equation is 283 + 334 / 167.0 result is 285
equation is 896 * 363 result is 325248
equation is 595 / 403 result is 1
equation is 860 + 493 / 246.5 result is 862
equation is 309 * 510 result is 157590
equation is 863 / 109 - 54.5 result is -46
equation is 360 + 482 - 241.0 result is 601
equation is 695 / 846 result is 0
equation is 703 - 784 - 392.0 result is -473
equation is 266 / 272 * 136.0 result is 133
equation is 806 * 627 + 313.5 result is 505675
equation is 10

equation is 925 * 463 / 231.5 result is 1850
equation is 393 * 42 result is 16506
equation is 390 / 407 result is 0
equation is 15 / 768 / 384.0 result is 0
equation is 490 + 517 result is 1007
equation is 561 - 727 result is -166
equation is 876 + 689 / 344.5 result is 878
equation is 376 + 851 - 425.5 result is 801
equation is 485 / 522 / 261.0 result is 0
equation is 565 / 923 result is 0
equation is 363 / 692 result is 0
equation is 496 / 426 result is 1
equation is 753 - 643 + 321.5 result is 431
equation is 876 / 504 result is 1
equation is 739 / 393 + 196.5 result is 198
equation is 291 / 450 + 225.0 result is 225
equation is 323 / 633 + 316.5 result is 317
equation is 603 * 231 - 115.5 result is 139177
equation is 632 + 400 result is 1032
equation is 816 / 614 * 307.0 result is 408
equation is 989 - 856 result is 133
equation is 473 + 116 result is 589
equation is 461 - 615 result is -154
equation is 689 + 622 result is 1311
equation is 875 * 3 result is 2625
equation is 793 / 

equation is 273 * 759 result is 207207
equation is 864 + 981 / 490.5 result is 866
equation is 227 / 99 - 49.5 result is -47
equation is 920 - 53 - 26.5 result is 840
equation is 340 * 553 result is 188020
equation is 919 - 161 * 80.5 result is -12041
equation is 862 / 269 result is 3
equation is 849 + 51 - 25.5 result is 874
equation is 558 + 688 result is 1246
equation is 244 * 480 result is 117120
equation is 888 + 278 result is 1166
equation is 654 / 268 / 134.0 result is 0
equation is 160 - 796 / 398.0 result is 158
equation is 489 - 88 / 44.0 result is 487
equation is 966 - 474 * 237.0 result is -111372
equation is 867 / 137 / 68.5 result is 0
equation is 621 * 338 result is 209898
equation is 474 + 983 * 491.5 result is 483618
equation is 792 * 297 result is 235224
equation is 917 / 798 result is 1
equation is 752 + 905 result is 1657
equation is 7 / 288 / 144.0 result is 0
equation is 924 + 703 result is 1627
equation is 683 / 462 result is 1
equation is 836 / 473 result is 1
e

equation is 789 * 501 result is 395289
equation is 983 + 991 result is 1974
equation is 421 * 135 result is 56835
equation is 181 * 991 result is 179371
equation is 575 - 607 result is -32
equation is 595 - 390 + 195.0 result is 400
equation is 762 * 992 result is 755904
equation is 211 + 247 / 123.5 result is 213
equation is 609 * 605 result is 368445
equation is 83 / 479 / 239.5 result is 0
equation is 313 - 12 result is 301
equation is 935 * 984 / 492.0 result is 1870
equation is 466 * 326 - 163.0 result is 151753
equation is 260 - 325 - 162.5 result is -227
equation is 432 + 344 result is 776
equation is 264 + 980 result is 1244
equation is 358 - 290 result is 68
equation is 622 / 811 / 405.5 result is 0
equation is 438 * 759 - 379.5 result is 332062
equation is 395 - 978 result is -583
equation is 910 + 918 result is 1828
equation is 290 / 252 result is 1
equation is 698 - 319 result is 379
equation is 187 / 763 result is 0
equation is 699 + 476 result is 1175
equation is 344 + 45

In [9]:
import os
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

model = build_model()

model.summary()
print()

# Let's print some predictions now to get a feeling for the equations
print()
print_example_predictions(5, model, x_test, y_test)
print()

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

try:
    model.fit(
        x_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(x_test, y_test),
        verbose=2,
        callbacks=[
            ModelCheckpoint(
                'model300.h5',
                save_best_only=True,
            ),
            TensorBoard(logdir, histogram_freq=1)
        ]
    )
except KeyboardInterrupt:
    print('\nCaught SIGINT\n')

# Load weights achieving best val_loss from training:
model.load_weights('model300.h5')

print_example_predictions(20, model, x_test, y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40)               6080      
 l)                                                              
                                                                 
 repeat_vector (RepeatVector  (None, 12, 40)           0         
 )                                                               
                                                                 
 bidirectional_1 (Bidirectio  (None, 12, 40)           9760      
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 12, 17)           697       
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 12, 17)            0

/Users/nehal/Library/Python/3.8/lib/python/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-05-08 21:46:53.000774: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz




Examples:
990 / 630  = ***---------   (expected:           1 )
130 / 650 / 325.0  = **6666666666   (expected:           0 )
386 * 681 * 340.5  = *****2222222   (expected:    89505873 )
6 * 711  = 111116666666   (expected:        4266 )
94 / 12 * 6.0  =  -----------   (expected:          47 )

Epoch 1/300
106/106 - 8s - loss: 1.0730 - accuracy: 0.6886 - val_loss: 0.8022 - val_accuracy: 0.7230 - 8s/epoch - 75ms/step
Epoch 2/300
106/106 - 2s - loss: 0.7571 - accuracy: 0.7308 - val_loss: 0.7248 - val_accuracy: 0.7306 - 2s/epoch - 20ms/step
Epoch 3/300
106/106 - 2s - loss: 0.7078 - accuracy: 0.7378 - val_loss: 0.7019 - val_accuracy: 0.7334 - 2s/epoch - 21ms/step
Epoch 4/300
106/106 - 2s - loss: 0.6855 - accuracy: 0.7432 - val_loss: 0.6865 - val_accuracy: 0.7449 - 2s/epoch - 21ms/step
Epoch 5/300
106/106 - 2s - loss: 0.6666 - accuracy: 0.7481 - val_loss: 0.6624 - val_accuracy: 0.7497 - 2s/epoch - 22ms/step
Epoch 6/300
106/106 - 2s - loss: 0.6531 - accuracy: 0.7515 - val_loss: 0.6617 - val_

Epoch 65/300
106/106 - 2s - loss: 0.4372 - accuracy: 0.8248 - val_loss: 0.4531 - val_accuracy: 0.8163 - 2s/epoch - 22ms/step
Epoch 66/300
106/106 - 2s - loss: 0.4451 - accuracy: 0.8218 - val_loss: 0.4590 - val_accuracy: 0.8155 - 2s/epoch - 23ms/step
Epoch 67/300
106/106 - 3s - loss: 0.4373 - accuracy: 0.8256 - val_loss: 0.4609 - val_accuracy: 0.8142 - 3s/epoch - 24ms/step
Epoch 68/300
106/106 - 3s - loss: 0.4389 - accuracy: 0.8243 - val_loss: 0.4518 - val_accuracy: 0.8186 - 3s/epoch - 26ms/step
Epoch 69/300
106/106 - 2s - loss: 0.4328 - accuracy: 0.8265 - val_loss: 0.4501 - val_accuracy: 0.8190 - 2s/epoch - 23ms/step
Epoch 70/300
106/106 - 2s - loss: 0.4311 - accuracy: 0.8274 - val_loss: 0.4521 - val_accuracy: 0.8183 - 2s/epoch - 22ms/step
Epoch 71/300
106/106 - 2s - loss: 0.4318 - accuracy: 0.8269 - val_loss: 0.4546 - val_accuracy: 0.8169 - 2s/epoch - 21ms/step
Epoch 72/300
106/106 - 2s - loss: 0.4350 - accuracy: 0.8257 - val_loss: 0.4578 - val_accuracy: 0.8191 - 2s/epoch - 23ms/step


Epoch 131/300
106/106 - 2s - loss: 0.3959 - accuracy: 0.8413 - val_loss: 0.4222 - val_accuracy: 0.8309 - 2s/epoch - 19ms/step
Epoch 132/300
106/106 - 2s - loss: 0.3981 - accuracy: 0.8408 - val_loss: 0.4270 - val_accuracy: 0.8291 - 2s/epoch - 19ms/step
Epoch 133/300
106/106 - 2s - loss: 0.3973 - accuracy: 0.8413 - val_loss: 0.4252 - val_accuracy: 0.8282 - 2s/epoch - 19ms/step
Epoch 134/300
106/106 - 2s - loss: 0.3945 - accuracy: 0.8417 - val_loss: 0.4244 - val_accuracy: 0.8293 - 2s/epoch - 19ms/step
Epoch 135/300
106/106 - 2s - loss: 0.3928 - accuracy: 0.8426 - val_loss: 0.4208 - val_accuracy: 0.8311 - 2s/epoch - 19ms/step
Epoch 136/300
106/106 - 2s - loss: 0.3908 - accuracy: 0.8432 - val_loss: 0.4216 - val_accuracy: 0.8312 - 2s/epoch - 18ms/step
Epoch 137/300
106/106 - 2s - loss: 0.3889 - accuracy: 0.8445 - val_loss: 0.4210 - val_accuracy: 0.8314 - 2s/epoch - 23ms/step
Epoch 138/300
106/106 - 2s - loss: 0.3890 - accuracy: 0.8444 - val_loss: 0.4324 - val_accuracy: 0.8284 - 2s/epoch - 19

106/106 - 2s - loss: 0.3787 - accuracy: 0.8477 - val_loss: 0.4178 - val_accuracy: 0.8335 - 2s/epoch - 19ms/step
Epoch 197/300
106/106 - 2s - loss: 0.3799 - accuracy: 0.8468 - val_loss: 0.4161 - val_accuracy: 0.8337 - 2s/epoch - 18ms/step
Epoch 198/300
106/106 - 2s - loss: 0.3785 - accuracy: 0.8477 - val_loss: 0.4186 - val_accuracy: 0.8326 - 2s/epoch - 18ms/step
Epoch 199/300
106/106 - 2s - loss: 0.3795 - accuracy: 0.8483 - val_loss: 0.4176 - val_accuracy: 0.8333 - 2s/epoch - 18ms/step
Epoch 200/300
106/106 - 2s - loss: 0.3778 - accuracy: 0.8480 - val_loss: 0.4099 - val_accuracy: 0.8365 - 2s/epoch - 18ms/step
Epoch 201/300
106/106 - 2s - loss: 0.3739 - accuracy: 0.8499 - val_loss: 0.4160 - val_accuracy: 0.8339 - 2s/epoch - 18ms/step
Epoch 202/300
106/106 - 2s - loss: 0.3802 - accuracy: 0.8471 - val_loss: 0.4123 - val_accuracy: 0.8349 - 2s/epoch - 18ms/step
Epoch 203/300
106/106 - 2s - loss: 0.3792 - accuracy: 0.8475 - val_loss: 0.4120 - val_accuracy: 0.8353 - 2s/epoch - 18ms/step
Epoch 

Epoch 262/300
106/106 - 2s - loss: 0.3689 - accuracy: 0.8513 - val_loss: 0.4087 - val_accuracy: 0.8363 - 2s/epoch - 15ms/step
Epoch 263/300
106/106 - 2s - loss: 0.3655 - accuracy: 0.8526 - val_loss: 0.4087 - val_accuracy: 0.8366 - 2s/epoch - 15ms/step
Epoch 264/300
106/106 - 2s - loss: 0.3658 - accuracy: 0.8524 - val_loss: 0.4138 - val_accuracy: 0.8365 - 2s/epoch - 15ms/step
Epoch 265/300
106/106 - 2s - loss: 0.3667 - accuracy: 0.8524 - val_loss: 0.4151 - val_accuracy: 0.8369 - 2s/epoch - 15ms/step
Epoch 266/300
106/106 - 2s - loss: 0.3666 - accuracy: 0.8527 - val_loss: 0.4123 - val_accuracy: 0.8361 - 2s/epoch - 15ms/step
Epoch 267/300
106/106 - 2s - loss: 0.3638 - accuracy: 0.8532 - val_loss: 0.4111 - val_accuracy: 0.8368 - 2s/epoch - 17ms/step
Epoch 268/300
106/106 - 2s - loss: 0.3756 - accuracy: 0.8498 - val_loss: 0.4213 - val_accuracy: 0.8348 - 2s/epoch - 15ms/step
Epoch 269/300
106/106 - 2s - loss: 0.3682 - accuracy: 0.8520 - val_loss: 0.4178 - val_accuracy: 0.8326 - 2s/epoch - 15

In [10]:
def predict(model, equation):
    """
    Given a model and an equation string, returns the predicted result.
    """
    x = np.zeros((1, MAX_EQUATION_LENGTH, N_FEATURES), dtype=np.bool)
    equation += '\0'

    for t, char in enumerate(equation):
        x[0, t, CHAR_TO_INDEX[char]] = 1

    predictions = model.predict(x)
    return one_hot_to_string(predictions[0])[:-1]

In [19]:
predict(model, '50 + 30')

/var/folders/k7/dvm_qjfs0wd10b19j7jnj8rw0000gn/T/ipykernel_19953/3076454189.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((1, MAX_EQUATION_LENGTH, N_FEATURES), dtype=np.bool)


'        158'